## set running directory

In [1]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

In [2]:
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir


DATA_PATH = os.path.join(os.environ['RNB_PLANNING_DIR'], "data")
try_mkdir(DATA_PATH)

TEST_DATA_PATH = os.path.join(DATA_PATH, "stowing-deep")
try_mkdir(TEST_DATA_PATH)

In [3]:
VISUALIZE = True
PLAY_RESULT = False
CLEARANCE = 1e-3
TOOL_RPY = (-np.pi/2,0,0)
TIMEOUT_MOTION = 5
MAX_TIME = 100


ROBOT_TYPE = RobotType.indy7gripper

if ROBOT_TYPE in [RobotType.indy7, RobotType.indy7gripper]:
    ROBOT_NAME = "indy0"
    TOOL_LINK = "indy0_tcp"
    TOOL_NAME = "grip0"
    TOOL_XYZ = (0,0,0.14)
    HOME_POSE = (0,0,0,0,0,0)
    GRIP_DEPTH = 0.05
    
elif ROBOT_TYPE == RobotType.panda:
    ROBOT_NAME = "panda0"
    TOOL_LINK = "panda0_hand"
    TOOL_NAME = "grip0"
    TOOL_XYZ = (0,0,0.112)
    HOME_POSE = (0,-0.3,0,-0.5,0,2.5,0)
    GRIP_DEPTH = 0.03
    
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, ROBOT_TYPE.name)
try_mkdir(ROBOT_DATA_ROOT)

DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, get_now())
# DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, '20210907-173519')
try_mkdir(DATASET_PATH)
print("")
print("DATASET_PATH: {}".format(DATASET_PATH))
print("")


DATASET_PATH: /home/rnb/Projects/rnb-planning/data/stowing-deep/indy7gripper/20210917-113211



## init combined robot config

In [4]:

from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, ROBOT_TYPE, None,
                INDY_IP)]
              , connection_list=[False])

from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")

connection command:
indy0: False


## get ghnd with detected robot config

In [5]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {ROBOT_NAME: ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob, start_rviz=VISUALIZE)
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


In [6]:
from pkg.utils.joint_utils import get_tf
shoulder_link = gscene.urdf_content.joint_map[gscene.joint_names[1]].child
shoulder_height = get_tf(shoulder_link, HOME_DICT, gscene.urdf_content)[2,3]

## add environment

In [7]:
from pkg.geometry.geometry import *
gtems_robot = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=False, collision=True)

## init planning scene

In [8]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

## Register binders

In [9]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [10]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name=TOOL_LINK, 
                 dims=(0.01,)*3, center=TOOL_XYZ, rpy=TOOL_RPY, color=(1,0,0,1), display=True, collision=False, fixed=True)
gripper = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))

Please create a subscriber to the marker


## ui

In [11]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


# Object Classes

In [12]:
from pkg.planning.pddlstream.scene_building import *

   Use a production WSGI server instead.
 * Debug mode: off


# Difficult sampling setting

In [13]:
if ROBOT_TYPE == RobotType.indy7gripper:
    WorkPlane.DIM_MIN = (0.6, 0.8, 0.05)
    WorkPlane.DIM_MAX = (0.6, 0.8, 0.05)
    
elif ROBOT_TYPE == RobotType.panda:
    WorkPlane.DIM_MIN = (0.4, 0.6, 0.05)
    WorkPlane.DIM_MAX = (0.4, 0.6, 0.05)

    WorkPlane.RTH_MAX = (0.4, np.pi/2, 0.4)
    WorkPlane.RTH_MIN = (0.3, -np.pi/2, -0.1)
    
    SideBox.RTH_MAX = (0.4, np.pi/2, 0.4)
    SideBox.RTH_MIN = (0.3, -np.pi/2, -0.1)
else:
    raise(NotImplementedError("Undefined robot type"))

Pole.DIM_MIN = (0.2, 0.2, 4)
Pole.DIM_MAX = (0.2, 0.2, 4)

PlaneObject.GTYPE = GEOTYPE.CYLINDER
PlaneObject.DIM_MAX = (0.05,0.05,0.2)
PlaneObject.DIM_MIN = (0.05,0.05,0.2)


In [14]:

def sample_difficult():
    gscene.clear_link("base_link")
    gscene.update_markers_all()
    gtimer = GlobalTimer.instance()
    gtimer.reset(stack=True)
    gtimer.tic("full_loop")
    for i_s in range(SAMPLE_NUM):
        ## add floor, ceiling
        floor = Floor(gscene, "floor")
        gscene.create_safe(gtype=GEOTYPE.BOX, name="base", link_name="base_link", 
                           dims=(0.2, 0.2, -floor.RTH[2]), center=(0,0,floor.RTH[2]/2), rpy=(0,0,0), 
                           color=floor.geometry.color, display=True, collision=True, fixed=True)
        # ceiling = Ceiling(gscene, "ceiling")

        ## set workplane
        wp = WorkPlane(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
        pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

        gp_colliding = True
        while gp_colliding:
            ## set goalplane
            gp = SideBox(gscene, "gp", floor_height=wp.RTH[2]+wp.RTH[2]/2)
            gp_colliding = wp.is_overlapped_with(gp.geometry) or gp.is_overlapped_with(wp.geometry)
        pscene.create_binder(bname="gp", gname="gp", _type=PlacePlane, point=None)

        for i_obs in range(Nmax_obs):
            colliding = True
            while colliding:
                pole = random.choice([Pole, Bar])(gscene, "obs"+str(i_obs))
                colliding = (wp.is_overlapped_with(pole.geometry, margin=OBS_MARGIN) 
                             or gp.is_overlapped_with(pole.geometry, margin=OBS_MARGIN))

        ## add object
        obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp, GRIP_DEPTH=GRIP_DEPTH, CLEARANCE=CLEARANCE)

        gtem_args = gscene.get_gtem_args()
        obj_args = {obj.name: {"GRIP_DEPTH": obj.GRIP_DEPTH, "DIM": obj.DIM, "CLEARANCE": CLEARANCE, "gname": obj.geometry.name} 
                    for obj in obj_list}

        save_pickle(os.path.join(DATASET_PATH, "data_"+f_option+"_%02d.pkl"%i_s), 
                    {"gtem_args":gtem_args, "obj_args": obj_args})
        gscene.update_markers_all()
        print("============ STEP ({} / {}) / Elapsed: {} s / ETA: {} s =============".format(i_s, SAMPLE_NUM,
                                         *np.round(np.divide(gtimer.eta("full_loop", i_s+1, SAMPLE_NUM), 1000), 1)))
        time.sleep(0.2)
#         raw_input("enter")

# sample difficult 1-0

In [15]:
Nmax_obj = 1
f_option = "obj_1c_obs0"
Nmax_obs = 0

SAMPLE_NUM = 100

OBS_MARGIN = 0.1

SHOW_PERIOD = 0.01


sample_difficult()

============ STEP (0 / 100) / Elapsed: 0.1 s / ETA: 8.9 s =============
============ STEP (1 / 100) / Elapsed: 0.6 s / ETA: 32.2 s =============
============ STEP (2 / 100) / Elapsed: 1.0 s / ETA: 33.0 s =============
============ STEP (3 / 100) / Elapsed: 1.3 s / ETA: 33.3 s =============
============ STEP (4 / 100) / Elapsed: 1.7 s / ETA: 33.6 s =============
============ STEP (5 / 100) / Elapsed: 2.4 s / ETA: 39.6 s =============
============ STEP (6 / 100) / Elapsed: 2.7 s / ETA: 38.8 s =============
============ STEP (7 / 100) / Elapsed: 3.1 s / ETA: 39.2 s =============
============ STEP (8 / 100) / Elapsed: 3.6 s / ETA: 40.2 s =============
============ STEP (9 / 100) / Elapsed: 4.0 s / ETA: 39.7 s =============
============ STEP (10 / 100) / Elapsed: 4.3 s / ETA: 39.2 s =============
============ STEP (11 / 100) / Elapsed: 4.7 s / ETA: 39.4 s =============
============ STEP (12 / 100) / Elapsed: 5.1 s / ETA: 39.6 s =============
============ STEP (13 / 100) / Elapsed: 5.6 s / E

# sample difficult 1-3

In [16]:
Nmax_obj = 1
f_option = "obj_1c_obs3"
Nmax_obs = 3

SAMPLE_NUM = 100

OBS_MARGIN = 0.1

SHOW_PERIOD = 0.01


sample_difficult()

============ STEP (0 / 100) / Elapsed: 0.2 s / ETA: 22.6 s =============
============ STEP (1 / 100) / Elapsed: 0.8 s / ETA: 42.2 s =============
============ STEP (2 / 100) / Elapsed: 1.4 s / ETA: 47.0 s =============
============ STEP (3 / 100) / Elapsed: 2.3 s / ETA: 56.7 s =============
============ STEP (4 / 100) / Elapsed: 2.7 s / ETA: 54.9 s =============
============ STEP (5 / 100) / Elapsed: 3.3 s / ETA: 54.4 s =============
============ STEP (6 / 100) / Elapsed: 3.8 s / ETA: 54.2 s =============
============ STEP (7 / 100) / Elapsed: 4.2 s / ETA: 52.6 s =============
============ STEP (8 / 100) / Elapsed: 5.1 s / ETA: 56.8 s =============
============ STEP (9 / 100) / Elapsed: 5.8 s / ETA: 57.8 s =============
============ STEP (10 / 100) / Elapsed: 6.3 s / ETA: 57.2 s =============
============ STEP (11 / 100) / Elapsed: 6.8 s / ETA: 56.3 s =============
============ STEP (12 / 100) / Elapsed: 7.1 s / ETA: 54.9 s =============
============ STEP (13 / 100) / Elapsed: 7.7 s / 

# sample difficult 1-5

In [17]:
Nmax_obj = 1
f_option = "obj_1c_obs5"
Nmax_obs = 5

SAMPLE_NUM = 100

OBS_MARGIN = 0.1

SHOW_PERIOD = 0.01


sample_difficult()

============ STEP (0 / 100) / Elapsed: 0.4 s / ETA: 40.4 s =============
============ STEP (1 / 100) / Elapsed: 1.3 s / ETA: 63.2 s =============
============ STEP (2 / 100) / Elapsed: 2.9 s / ETA: 95.9 s =============
============ STEP (3 / 100) / Elapsed: 3.5 s / ETA: 86.8 s =============
============ STEP (4 / 100) / Elapsed: 4.0 s / ETA: 80.1 s =============
============ STEP (5 / 100) / Elapsed: 4.7 s / ETA: 77.7 s =============
============ STEP (6 / 100) / Elapsed: 5.2 s / ETA: 73.8 s =============
============ STEP (7 / 100) / Elapsed: 5.6 s / ETA: 70.2 s =============
============ STEP (8 / 100) / Elapsed: 6.3 s / ETA: 69.7 s =============
============ STEP (9 / 100) / Elapsed: 6.9 s / ETA: 69.2 s =============
============ STEP (10 / 100) / Elapsed: 7.4 s / ETA: 67.1 s =============
============ STEP (11 / 100) / Elapsed: 8.0 s / ETA: 66.4 s =============
============ STEP (12 / 100) / Elapsed: 8.5 s / ETA: 65.8 s =============
============ STEP (13 / 100) / Elapsed: 9.3 s / 

# sample difficult 1-7

In [18]:
Nmax_obj = 1
f_option = "obj_1c_obs7"
Nmax_obs = 7

SAMPLE_NUM = 100

OBS_MARGIN = 0.1

SHOW_PERIOD = 0.01


sample_difficult()

============ STEP (0 / 100) / Elapsed: 0.1 s / ETA: 7.9 s =============
============ STEP (1 / 100) / Elapsed: 1.4 s / ETA: 72.0 s =============
============ STEP (2 / 100) / Elapsed: 2.7 s / ETA: 89.5 s =============
============ STEP (3 / 100) / Elapsed: 3.1 s / ETA: 78.7 s =============
============ STEP (4 / 100) / Elapsed: 4.0 s / ETA: 80.1 s =============
============ STEP (5 / 100) / Elapsed: 4.7 s / ETA: 78.4 s =============
============ STEP (6 / 100) / Elapsed: 5.2 s / ETA: 74.3 s =============
============ STEP (7 / 100) / Elapsed: 6.0 s / ETA: 74.5 s =============
============ STEP (8 / 100) / Elapsed: 6.6 s / ETA: 73.1 s =============
============ STEP (9 / 100) / Elapsed: 7.1 s / ETA: 71.1 s =============
============ STEP (10 / 100) / Elapsed: 7.8 s / ETA: 71.2 s =============
============ STEP (11 / 100) / Elapsed: 8.5 s / ETA: 70.5 s =============
============ STEP (12 / 100) / Elapsed: 9.0 s / ETA: 69.4 s =============
============ STEP (13 / 100) / Elapsed: 9.7 s / E

## sample difficult 3-3

In [19]:
Nmax_obj = 3
f_option = "obj_3c_obs3"
Nmax_obs = 3

SAMPLE_NUM = 100

OBS_MARGIN = 0.1

SHOW_PERIOD = 0.01


sample_difficult()

============ STEP (0 / 100) / Elapsed: 0.3 s / ETA: 25.1 s =============
============ STEP (1 / 100) / Elapsed: 1.3 s / ETA: 66.8 s =============
============ STEP (2 / 100) / Elapsed: 1.9 s / ETA: 64.3 s =============
============ STEP (3 / 100) / Elapsed: 2.5 s / ETA: 61.4 s =============
============ STEP (4 / 100) / Elapsed: 3.0 s / ETA: 59.2 s =============
============ STEP (5 / 100) / Elapsed: 3.9 s / ETA: 65.0 s =============
============ STEP (6 / 100) / Elapsed: 4.5 s / ETA: 64.1 s =============
============ STEP (7 / 100) / Elapsed: 5.0 s / ETA: 62.4 s =============
============ STEP (8 / 100) / Elapsed: 5.5 s / ETA: 60.6 s =============
============ STEP (9 / 100) / Elapsed: 5.9 s / ETA: 58.9 s =============
============ STEP (10 / 100) / Elapsed: 6.3 s / ETA: 57.7 s =============
============ STEP (11 / 100) / Elapsed: 7.0 s / ETA: 58.0 s =============
============ STEP (12 / 100) / Elapsed: 7.5 s / ETA: 57.3 s =============
============ STEP (13 / 100) / Elapsed: 8.3 s / 

## sample difficult 3-5

In [20]:
Nmax_obj = 3
f_option = "obj_3c_obs5"
Nmax_obs = 5

SAMPLE_NUM = 100

OBS_MARGIN = 0.1

SHOW_PERIOD = 0.01


sample_difficult()

============ STEP (0 / 100) / Elapsed: 0.2 s / ETA: 15.2 s =============
============ STEP (1 / 100) / Elapsed: 0.8 s / ETA: 38.0 s =============
============ STEP (2 / 100) / Elapsed: 1.2 s / ETA: 41.1 s =============
============ STEP (3 / 100) / Elapsed: 1.8 s / ETA: 45.3 s =============
============ STEP (4 / 100) / Elapsed: 3.0 s / ETA: 59.2 s =============
============ STEP (5 / 100) / Elapsed: 3.5 s / ETA: 58.6 s =============
============ STEP (6 / 100) / Elapsed: 4.0 s / ETA: 57.4 s =============
============ STEP (7 / 100) / Elapsed: 4.5 s / ETA: 56.8 s =============
============ STEP (8 / 100) / Elapsed: 5.0 s / ETA: 56.0 s =============
============ STEP (9 / 100) / Elapsed: 5.6 s / ETA: 55.8 s =============
============ STEP (10 / 100) / Elapsed: 6.1 s / ETA: 55.1 s =============
============ STEP (11 / 100) / Elapsed: 6.6 s / ETA: 54.8 s =============
============ STEP (12 / 100) / Elapsed: 7.1 s / ETA: 54.6 s =============
============ STEP (13 / 100) / Elapsed: 7.7 s / 

## sample difficult 3-7

In [21]:
Nmax_obj = 3
f_option = "obj_3c_obs7"
Nmax_obs = 7

SAMPLE_NUM = 100

OBS_MARGIN = 0.1

SHOW_PERIOD = 0.01


sample_difficult()

============ STEP (0 / 100) / Elapsed: 0.2 s / ETA: 23.1 s =============
============ STEP (1 / 100) / Elapsed: 0.9 s / ETA: 44.3 s =============
============ STEP (2 / 100) / Elapsed: 1.4 s / ETA: 48.2 s =============
============ STEP (3 / 100) / Elapsed: 2.0 s / ETA: 50.1 s =============
============ STEP (4 / 100) / Elapsed: 2.5 s / ETA: 50.5 s =============
============ STEP (5 / 100) / Elapsed: 3.2 s / ETA: 52.8 s =============
============ STEP (6 / 100) / Elapsed: 3.8 s / ETA: 54.8 s =============
============ STEP (7 / 100) / Elapsed: 4.4 s / ETA: 54.5 s =============
============ STEP (8 / 100) / Elapsed: 5.4 s / ETA: 60.4 s =============
============ STEP (9 / 100) / Elapsed: 6.0 s / ETA: 60.2 s =============
============ STEP (10 / 100) / Elapsed: 6.6 s / ETA: 59.6 s =============
============ STEP (11 / 100) / Elapsed: 7.2 s / ETA: 59.6 s =============
============ STEP (12 / 100) / Elapsed: 7.8 s / ETA: 60.1 s =============
============ STEP (13 / 100) / Elapsed: 8.4 s / 